# Addaction Analysis

Addaction have contacted us off the back of the blog on the impact on NCSO on local authorties and charities. We estimated in our blog that the impact of NCSO on Addaction is an additional £226,144.28 on top of a "usual" tariff cost of £47 257.92 giving a total of £273,402.2 in 2018. Addaction have estimated the impact on them is £403,474.59.

In [2]:
import pandas as pd
import numpy as np
from ebmdatalab import bq

In [ ]:
## SQL query to extract additonal costs of NHS price concessions. 
## This is a modfied version of the SQL query from OpenPrescribing.net NHS price concession calculator site i.e. done at a "organisational level" 
sql = '''SELECT
  ncso.date AS month,
  product.bnf_code AS bnf_code,
  product.name AS product_name,
  rx.quantity AS quantity,
  rx.pct AS pct,
  ccg.org_type,
  ccg.name,
  dt.price_pence
    * rx.quantity
    * CASE WHEN
        -- For some presentations "quantity" means "number of packs" rather
        -- than e.g. tablets. In these cases we don't want to divide by the
        -- quantity value of a pack. This is implemented via a flag in our
        -- databse but this data isn't in BiqQuery so we just have a hardcoded
        -- list of BNF codes here
        product.bnf_code in ('0206010F0AACJCJ')
      THEN
        1
      ELSE
        1 / vmpp.qtyval
      END
    -- This is the "discount factor" which applies the National Average 7.2%
    -- discount to estimate Actual Cost from Net Ingredient Cost and also
    -- converts figures from pence to pounds
    * 0.00928
    AS tariff_cost,
  COALESCE(ncso.price_concession_pence - dt.price_pence, 0)
    * rx.quantity
    * CASE WHEN
        product.bnf_code in ('0206010F0AACJCJ')
      THEN
        1
      ELSE
        1 / vmpp.qtyval
      END
    * 0.00928
    AS additional_cost,
  ncso.date != rx.month AS is_projection
FROM
  dmd.ncsoconcession AS ncso
JOIN
  dmd.tariffprice AS dt
ON
  ncso.vmpp = dt.vmpp AND ncso.date = dt.date
JOIN
  dmd.product AS product
ON
  dt.product=product.dmdid
JOIN
  dmd.vmpp AS vmpp
ON
  vmpp.vppid=ncso.vmpp
JOIN
  hscic.presentation AS presentation
ON
  presentation.bnf_code = product.bnf_code
JOIN
  hscic.prescribing AS rx
ON
  rx.bnf_code = product.bnf_code
JOIN
  hscic.ccgs AS ccg #this joins to our CCG organisational data
ON
  rx.pct = ccg.code
AND
-- Where we have prescribing data for the corresponding month we use
-- that, otherwise we use the latest month of prescribing data to
-- produce an estimate
(
  rx.month = ncso.date
  OR
  (
    -- This should be set to the latest date for which we have prescribing
    -- data
    rx.month = TIMESTAMP('2019-01-01')
    AND
    ncso.date > rx.month
  )
)'''
    
df_ncso = bq.cached_read(sql, csv_path='df_ncso.zip', use_cache=True)  # add `use_cache=False` to override
df_ncso.head()

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas_gbq\gbq.py:556: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  "verbosity", FutureWarning, stacklevel=1)


In [3]:
## import "independent providers" from NHS Digital ODS codes as this is where addaction is listed.
independent_providers = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\jupyter-notebooks\nonCCG Concessions\ephp.csv')
independent_providers.head(5)

,pct,name,national_grouping,high_level_health_geography,address line 1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,postcode,open_date,close_date
0,AA4,INTRAHEALTH LTD,Y54,Q74,"1ST FLOOR, WILLIAM BROWN CENTRE",MANOR WAY,NaN,PETERLEE,COUNTY DURHAM,SR8 5TW,20130401,NaN
1,AA5,COMPASS WELLBEING COMMUNITY INTEREST COMPANY,Y56,Q71,STEELS LANE HEALTH CENTRE,384-388 COMMERCIAL ROAD,NaN,LONDON,GREATER LONDON,E1 0LR,20130401,NaN
2,AA6,ASSISTED CONCEPTION UNIT LTD,Y56,Q71,LEYTONSTONE HOUSE,LEYTONSTONE,NaN,LONDON,GREATER LONDON,E11 1GA,20130401,NaN
3,AA7,LEYLAND PHYSIOTHERAPY,Y54,Q84,83 BOW LANE,NaN,NaN,LEYLAND,LANCASHIRE,PR25 4YB,20130401,NaN
4,AA8,SMART CJS,Y59,Q87,BUILDING B,KIRTLINGTON BUSINESS CENTRE,"SLADE FARM, KIRTLINGTON",KIDLINGTON,OXFORDSHIRE,OX5 3JA,20130401,NaN


In [ ]:
## adding in independent provider names to our prescribing dataset result from SQL
indopro = df_ncso.merge(independent_providers[['pct','name']],  how="outer", on='pct')
indopro.head()

In [ ]:
## clearing blanks from the datset
df2 = indopro.dropna(subset=['name_y'])
df2

In [ ]:
## filtering the dataset to only include results where Addaction or similar is in the independent provider name
addaction2 = df2[df2["name_y"].str.contains("addaction", case=False)]
addaction2.head()

In [ ]:
## Counting how many variations on the name Addaction are retured
addaction2["name_y"].unique()

There is only one variation of the name Addaction in our dataset therefore we can be reasonably confident that our impact is correct as per the NHS BSA primary care prescribing/reimbursement dataset.

In [ ]:
### see impact of NCSO in 2018
addaction2018 = addaction2.loc[(addaction2["month"]>="2018-01-01") & (addaction2["month"]<="2018-12-01")]
addaction2018

In [ ]:
### check impact for 2018
addaction2018.sum()

Now we will check the spend on Buprenorphine (BNF code = 0410030A0%) to see if Addaction are counting total impact instead of diferentiating between NCSO and "normal cost". The difference here is we are looking at all Buprenorphine products in all months of 2018. The previous analysis only look at products in months when they are NCSO.

In [ ]:
sql = '''SELECT
  extract(year from month) AS year,
  bnf_code AS bnfcode,
  bnf_name AS bnfname,
  SUM(items) AS total_items,
  SUM(quantity) AS total_qty,
  SUM(actual_cost) AS actual_cost
FROM
  ebmdatalab.hscic.normalised_prescribing_standard AS presc
WHERE
  bnf_code LIKE "0410030A0%"  
  and
  pct = "NI3" 
GROUP BY
  year,
  bnf_code,
  bnf_name'''
    
df_addaction = bq.cached_read(sql, csv_path='df_addaction.csv', use_cache=True)  # add `use_cache=False` to override
df_addaction.head()

In [ ]:
### grouping the total cost by year
totalcost = df_addaction.groupby(['year'])['actual_cost'].sum()
totalcost

In [ ]:
## graphing full years - i.e. get rid of 2010 and 2019 as incomplete
totalcostfullyearsonly = totalcost.drop([2010, 2019])
ax = totalcostfullyearsonly.plot(kind='line', title = "Total Cost £ of Buprenorphine at Addaction")
ax.set_ylim(0, 400000)


Addaction spent £352,194.52 on Buprenorphine  (BNF code = 0410030A0%) as per the prescribing file which is £51,280 short of their estimate of £403,474.59. We will now clarify/ask
-  What is the time frame? 18/19 has not finished so are they estimating or is the £403k a partial year spend?
-  What is the datset? for example is this invoiced?
-  Do they have other prescribing codes where the name is not similar to Addaction?
-  Do they pay through a "recharge" mechanism? e.g. a CCG pays and then invoices addaction